In [40]:
from sklearn.cluster import DBSCAN
import pandas as pd
import matplotlib.pyplot  as plt
import seaborn as sns
import numpy as np
import pickle
from numpy import dot
from numpy.linalg import norm

# 데이터 불러오기

In [28]:
df = pd.read_csv('/Users/heogeonhyeock/Downloads/데이터통합_수정.csv')
df.set_index("Unnamed: 0",inplace=True)
df.index.name = "index"
df

/Users/heogeonhyeock/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CORP_ID,REF_ORD_NO,BKG_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_CD,...,송화 -> 터미널,터미널 -> 수화,distance_송화,distance_수화,distance,일강수량(mm),rain,TIME(12),TIME(3),TIME(6)
index,,,,,,,,,,,,,,,,,,,,,
1,KX007,2021030365922951,KX007210303371080,7,20210303,35521,90001441,20900.0,1,4610261043_2,...,514547,5390,14.336226,270.612949,284.949175,3.1,1,3443,3693,1412
2,KX007,2021030365924361,KX007210303371081,7,20210303,35513,90001441,37900.0,1,4936599538_1,...,514547,5390,14.336226,294.612573,308.948799,6.0,1,3443,3693,1412
3,KX007,2021030365928921,KX007210303371205,7,20210303,35608,90001542,69800.0,1,3858391637_1,...,514547,513833,14.336226,32.840975,47.177201,0.0,0,3443,3693,1412
4,KX007,2021030365930451,KX007210303370967,7,20210303,35551,90001341,73700.0,1,4631307460,...,514547,513833,14.336226,64.281087,78.617314,0.0,0,3443,3693,1412
5,KX007,2021030365930451,KX007210303370967,7,20210303,35551,90001341,73700.0,2,2475138283,...,514547,513833,14.336226,64.281087,78.617314,0.0,0,3443,3693,1412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786343,KX007,2021063075245201,KX007210702671565,7,20210630,122001,90001622,125800.0,1,5519470982_18,...,528659,402678,14.336226,41.458529,55.794755,0.0,0,687,1361,2944
1786347,KX007,2021062954478421,KX007210702674788,7,20210629,122812,90001443,16900.0,1,5002901762_12,...,528659,402678,14.336226,28.415601,42.751828,0.0,0,687,1361,2944
1786348,KX007,2021063067849541,KX007210703706457,7,20210630,256,90001662,138000.0,1,5410093340,...,제주,제주,14.336226,432.124577,446.460804,0.0,0,687,1361,2944


## 제주 데이터 제거

In [29]:
df = df[~((df["CNEE_ADDR_1"] == "제주특별자치도") | (df["CNEE_ADDR_1"] == "제주"))]

## 계산에 이용할 컬럼 추출
    송화 -> 터미널 
    터미널 -> 수화 데이터타입이 object 이므로 int 형으로 변경

In [30]:
df = df[['TIME(12)','rain',"distance_송화","distance_수화",'distance',"송화 -> 터미널","터미널 -> 수화"]]
df = df.astype({'송화 -> 터미널' : "int", '터미널 -> 수화' : "int"})
df

,TIME(12),rain,distance_송화,distance_수화,distance,송화 -> 터미널,터미널 -> 수화
index,,,,,,,
1,3443,1,14.336226,270.612949,284.949175,514547,5390
2,3443,1,14.336226,294.612573,308.948799,514547,5390
3,3443,0,14.336226,32.840975,47.177201,514547,513833
4,3443,0,14.336226,64.281087,78.617314,514547,513833
5,3443,0,14.336226,64.281087,78.617314,514547,513833
...,...,...,...,...,...,...,...
1786341,687,0,14.336226,58.562049,72.898275,402678,302624
1786343,687,0,14.336226,41.458529,55.794755,528659,402678
1786347,687,0,14.336226,28.415601,42.751828,528659,402678


## 교통량 데이터 전처리

In [31]:
a_distance = np.array(df["distance_송화"])
b_distance = np.array(df["distance_수화"])
a_trafic = np.array(df["송화 -> 터미널"])
b_trafic = np.array(df["터미널 -> 수화"])

c = (a_distance * a_trafic + b_distance * b_trafic) / (a_distance + b_distance)
c

array([ 31006.46321009,  29016.53643474, 514049.970602  , ...,
       444923.96338209, 102546.67263984,  23662.68376844])

## feature 데이터 생성

In [32]:
df['교통량'] = c
feature = df[['TIME(12)', 'rain','distance',"교통량"]]
feature = feature.dropna(axis=0)
feature = feature.astype("int")
feature

,TIME(12),rain,distance,교통량
index,,,,
1,3443,1,284,31006
2,3443,1,308,29016
3,3443,0,47,514049
4,3443,0,78,513963
5,3443,0,78,513963
...,...,...,...,...
1786341,687,0,72,322300
1786343,687,0,55,435048
1786347,687,0,42,444923


## min_max 값 저장

In [7]:

min_max_value = []
for i in feature.columns:
    min_max_value.append(min(feature[i]))
    min_max_value.append(max(feature[i]))
with open("min_max_value.pickle2222","wb") as fw:
    pickle.dump(min_max_value, fw)


# 데이터 저장 및 불러오기

In [26]:
"""
with open("feature2222.pickle","wb") as fw:
    pickle.dump(feature, fw)
"""
with open("/Users/heogeonhyeock/Desktop/인빅_프로젝트/feature_result.pickle","rb") as fr:
    feature = pickle.load(fr)
feature

EOFError: Ran out of input

# 분류 모델

In [9]:
from sklearn.preprocessing import MinMaxScaler

test_data = np.array(MinMaxScaler().fit_transform(feature))
test_data

array([[0.13395324, 1.        , 0.60046189, 0.03223468],
       [0.13395324, 1.        , 0.65588915, 0.02841799],
       [0.13395324, 0.        , 0.05311778, 0.95867816],
       ...,
       [0.0267284 , 0.        , 0.04157044, 0.82609921],
       [0.0267284 , 0.        , 0.22170901, 0.16944351],
       [0.0267284 , 0.        , 0.64665127, 0.01814939]])

## 모델 적용

In [ ]:
# create model and prediction
model = DBSCAN(eps=0.1,n_jobs=4)
predict = model.fit_predict(test_data)
feature["predict"] = predict
feature

# 군집화 시각화

## 요약통계

In [ ]:
for i in feature.columns[:4]:
    feature[i] = (feature[i] - min(feature[i])) / (max(feature[i]) - min(feature[i]))
feature_result = feature.groupby("predict").mean()
feature_result

In [ ]:
"""
with open("feature_result2222.pickle","wb") as fw:
    pickle.dump(feature_result, fw)
"""

## 시각화

In [ ]:
sns.pairplot(feature,hue='predict')
plt.show()

In [ ]:
def visualize_cluster_plot(clusterobj, dataframe, label_name, iscenter=True):
    if iscenter :
        centers = clusterobj.cluster_centers_

    unique_labels = np.unique(dataframe[label_name].values)
    markers=['o', 's', '^', 'x', '*','<','>']
    isNoise=False

    for label in unique_labels:
        label_cluster = dataframe[dataframe[label_name]==label]
        if label == -1:
            cluster_legend = 'Noise'
            isNoise=True
        else :
            cluster_legend = 'Cluster '+str(label)

        plt.scatter(x=label_cluster['ftr1'], y=label_cluster['ftr2'], s=70,\
                    edgecolor='k', marker=markers[label], label=cluster_legend)

        if iscenter:
            center_x_y = centers[label]
            plt.scatter(x=center_x_y[0], y=center_x_y[1], s=250, color='white',
                        alpha=0.9, edgecolor='k', marker=markers[label])
            plt.scatter(x=center_x_y[0], y=center_x_y[1], s=70, color='k',\
                        edgecolor='k', marker='$%d$' % label)
    if isNoise:
        legend_loc='best'
    else: legend_loc='best'

#    plt.legend(loc=legend_loc)
    plt.show()

from sklearn.decomposition import PCA
# 2차원으로 시각화하기 위해 PCA n_componets=2로 피처 데이터 세트 변환
pca = PCA(n_components=4, random_state=0)
pca_transformed = pca.fit_transform(feature)
pca_transformed 

# visualize_cluster_2d( ) 함수는 ftr1, ftr2 컬럼을 좌표에 표현하므로 PCA 변환값을 해당 컬럼으로 생성
feature['ftr1'] = pca_transformed[:,0]
feature['ftr2'] = pca_transformed[:,1]
feature['ftr3'] = pca_transformed[:,2]
feature['ftr4'] = pca_transformed[:,3]

visualize_cluster_plot(model, feature, 'predict', iscenter=False)

# 예측 모델
    예측데이터의 데이터프레임 순서 : TIME(12) rain distance 교통량

In [157]:
with open("/Users/heogeonhyeock/Desktop/인빅_프로젝트/min_max_value.pickle","rb") as fr:
    min_max_value = pickle.load(fr)
with open("/Users/heogeonhyeock/Desktop/인빅_프로젝트/feature_result.pickle","rb") as fr:
    feature_result = pickle.load(fr)
feature_result =feature_result[2:]

In [152]:
# 테스트 데이터
df = feature[300000:400000]
df

,TIME(12),rain,distance,교통량
index,,,,
332928,0,0,72,522991
332929,0,0,53,523027
332930,0,0,90,126009
332931,0,0,90,126009
332932,0,0,77,522986
...,...,...,...,...
442876,193,0,47,457516
442877,193,1,72,449029
442878,193,0,70,449521


In [153]:
def IN_BIG(df):
        
    min_max_dict = {}
    for i in range(len(df.columns)):
        min_max_dict[df.columns[i]] = min_max_value[2*i:2*(i+1)]
    
    for col in df.columns:
        df[col] = (df[col] - min_max_dict[col][0]) / (min_max_dict[col][1] - min_max_dict[col][0])
    
    def cos_sim(A, B):
           return dot(A, B)/(norm(A)*norm(B))

    A = []
    for i in range(len(feature_result)):
        A.append(cos_sim(np.array(df),np.array(feature_result)[i]))
    df["predict"] = 1 + np.argmin(np.transpose(A), axis=1)
    return df

predic_df = IN_BIG(df)
predic_df

<ipython-input-153-1e86d70c99f2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = (df[col] - min_max_dict[col][0]) / (min_max_dict[col][1] - min_max_dict[col][0])
<ipython-input-153-1e86d70c99f2>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["predict"] = 1 + np.argmin(np.transpose(A), axis=1)


,TIME(12),rain,distance,교통량,predict
index,,,,,
332928,0.000000,0.0,0.143284,0.975828,2
332929,0.000000,0.0,0.086567,0.975897,2
332930,0.000000,0.0,0.197015,0.214444,2
332931,0.000000,0.0,0.197015,0.214444,2
332932,0.000000,0.0,0.158209,0.975819,2
...,...,...,...,...,...
442876,0.007552,0.0,0.068657,0.850252,2
442877,0.007552,1.0,0.143284,0.833974,2
442878,0.007552,0.0,0.137313,0.834918,2
